In [109]:
import pandas as pd
from pandas._libs.internals import defaultdict
from unidecode import unidecode
from collections.abc import Callable
import re
from typing import Optional
import uuid
import random

<h1>Defining Functions</h1>

CSV Cleaning Functions

In [110]:
# convert a csv into utf8 format
def convert_utf8(original_file_path: str, new_file_path: str):
    df = pd.read_csv(original_file_path, converters=defaultdict(lambda i: str))
    for column in df.columns:
        df[column] = df[column].apply(lambda x: unidecode(str(x)))
    df.to_csv(new_file_path, encoding='utf-8')

Single Entry Cleaning Functions
Note: Presented in recommended order of use for many, but not all, applications

In [111]:
# Remove special characters when dealing with words
# transforms removed characters into spaces, removes . and -
def remove_special_for_words(my_string: str):
    return re.sub(r"[^a-zA-Z0-9]+", ' ', my_string)

# Remove special characters when dealing with numbers
# No spaces, does not remove - and .
def remove_special_for_numbers(my_string: str):
    return re.sub(r"[^a-zA-Z0-9-.]+", '', my_string)

# Remove leading and trailing spaces
def truncate(my_string: str):
    return my_string.strip()

# Replace spaces with underscores
def snake_case(my_string: str):
    return my_string.replace(" ", "_")

# Make all letters lowercase
def lower_case(my_string: str):
    return my_string.lower()

def remove_spaces(my_string: str):
    return my_string.replace(" ", "")

# Get a deterministic hash of a string, dependent only on the seed 
# (for creating unique id columns which will always be consistent when fed the same data and seed, even if generated at different times)
def deterministic_uuid(data:str):
    random.seed(data)
    id = uuid.UUID(bytes=bytes(random.getrandbits(8) for _ in range(16)), version=4)
    id = str(id).replace("-", "")
    return id

Dataframe Cleaning Functions

In [112]:
# Apply a list of functions to the column headers
def clean_headers(df: pd.DataFrame, 
                  cleaning_functions_list:list[Callable]) -> pd.DataFrame:
    column_name_map = {item: item for item in df.columns}
    for item in column_name_map:
        for function in cleaning_functions_list:
            column_name_map[item] = function(column_name_map[item])
    df.rename(columns=column_name_map, inplace=True)
    return df

# Apply a list of functions to selected columns
def clean_columns(df: pd.DataFrame, 
                  selected_columns: list[str], 
                  cleaning_functions_list:list[Callable]) -> pd.DataFrame:
    for col in selected_columns:
        for function in cleaning_functions_list:
            df[col] = df[col].apply(lambda x: function(x))
    return df

# Apply a list of functions to every single entry in the dataframe
def clean_entries(df: pd.DataFrame, 
                  cleaning_functions_list:list[Callable]) -> pd.DataFrame:
    for function in cleaning_functions_list:
        df = df.apply(lambda x: function(x))
        return df
    
def cast_data_types(df: pd.DataFrame, names_to_types: dict) -> pd.DataFrame:
    return df.astype(names_to_types)

# Create a unique index column by taking a deterministic hash of values in selected identifier columns
def set_unique_index(df: pd.DataFrame, columns_to_hash: list[str], index_name="id", index_length_limit: Optional[int] = None):
    df[index_name] = list(map(lambda x: deterministic_uuid(''.join([str(col_value) for col_value in x]))[0:index_length_limit], df[columns_to_hash].values))
    df.set_index(index_name, inplace=True)
    return df


<h1>Cleaning a CSV</h1>

Read in a CSV

In [121]:
# load a dataframe
infile_path = "../data/raw/silver_shelby_county_properties_municipality_breakdown - s1_2019.csv"
df = pd.read_csv(infile_path)
df

,Board,Number of Contracts,Amount Billed Pilot (INL),Amount Billed Debt Service,Assessment,Tax Based on Assessment at 2021 Tax Rate,Difference,Percentage Total PILOT Billing
0,Downtown Memphis Commission,123,"$1,981,904.47","$1,226,006.18","$349,460,540","$14,153,151.87","-10,945,241.22",20.69%
1,EDGE - Shelby,243,"$2,856,659.78","$6,149,156.86","$673,220,031","$27,265,411.26","-18,259,594.62",58.08%
2,Industrial Development of Arlington,15,"$52,116.90","$114,566.56","$11,977,950","$485,106.98","-318,423.52",1.07%
3,Industrial Development of Bartlett,11,"$15,257.82","$146,630.61","$14,962,783","$605,992.71","-444,104.28",1.04%
4,Industrial Development of Collierville,14,"$774,048.02","$661,885.66","$70,132,723","$2,840,375.28","-1,404,441.60",9.26%
5,Industrial Development of Germantown,4,"$237,740.75","$96,081.97","$15,357,250","$621,968.63","-288,145.91",2.15%
6,Industrial Development of Millington,1,"$10,280.12","$79,383.24","$7,840,320","$317,532.96","-227,869.60",0.58%
7,Health & Education Board - Memphis,73,"$930,606.59",$0.00,"$128,138,985","$5,189,628.89","-4,259,022.30",6.00%
8,Health & Education Board - Shelby,8,"$133,533.27",$0.00,"$27,687,400","$1,121,339.70","-987,806.43",0.86%
9,Miscellaneous Contracts,10,"$35,460.00",$0.00,"$9,033,520","$365,857.56","-330,397.56",0.23%


In [122]:
# Clean the column headers
df = clean_headers(df, [remove_special_for_words, truncate, snake_case, lower_case])
df.columns

Index(['board', 'number_of_contracts', 'amount_billed_pilot_inl',
       'amount_billed_debt_service', 'assessment',
       'tax_based_on_assessment_at_2021_tax_rate', 'difference',
       'percentage_total_pilot_billing'],
      dtype='object')

In [115]:
# Clean the string columns
string_columns = ["board"]
df = clean_columns(df, string_columns, [remove_special_for_words, truncate, lower_case])
df

,board,number_of_contracts,amount_billed_pilot_inl,amount_billed_debt_service,assessment,tax_based_on_assessment_at_2021_tax_rate,difference,percentage_total_pilot_billing
0,downtown memphis commission,123,"$1,981,904.47","$1,226,006.18","$349,460,540","$14,153,151.87","-10,945,241.22",20.69%
1,edge shelby,243,"$2,856,659.78","$6,149,156.86","$673,220,031","$27,265,411.26","-18,259,594.62",58.08%
2,industrial development of arlington,15,"$52,116.90","$114,566.56","$11,977,950","$485,106.98","-318,423.52",1.07%
3,industrial development of bartlett,11,"$15,257.82","$146,630.61","$14,962,783","$605,992.71","-444,104.28",1.04%
4,industrial development of collierville,14,"$774,048.02","$661,885.66","$70,132,723","$2,840,375.28","-1,404,441.60",9.26%
5,industrial development of germantown,4,"$237,740.75","$96,081.97","$15,357,250","$621,968.63","-288,145.91",2.15%
6,industrial development of millington,1,"$10,280.12","$79,383.24","$7,840,320","$317,532.96","-227,869.60",0.58%
7,health education board memphis,73,"$930,606.59",$0.00,"$128,138,985","$5,189,628.89","-4,259,022.30",6.00%
8,health education board shelby,8,"$133,533.27",$0.00,"$27,687,400","$1,121,339.70","-987,806.43",0.86%
9,miscellaneous contracts,10,"$35,460.00",$0.00,"$9,033,520","$365,857.56","-330,397.56",0.23%


In [116]:
# Clean the numeric columns
numeric_columns = ['amount_billed_pilot_inl', 
                   'amount_billed_debt_service', 
                   'assessment',
                   'tax_based_on_assessment_at_2021_tax_rate', 
                   'difference',
                   'percentage_total_pilot_billing']
df = clean_columns(df, numeric_columns, [remove_special_for_numbers])
df

,board,number_of_contracts,amount_billed_pilot_inl,amount_billed_debt_service,assessment,tax_based_on_assessment_at_2021_tax_rate,difference,percentage_total_pilot_billing
0,downtown memphis commission,123,1981904.47,1226006.18,349460540,14153151.87,-10945241.22,20.69
1,edge shelby,243,2856659.78,6149156.86,673220031,27265411.26,-18259594.62,58.08
2,industrial development of arlington,15,52116.90,114566.56,11977950,485106.98,-318423.52,1.07
3,industrial development of bartlett,11,15257.82,146630.61,14962783,605992.71,-444104.28,1.04
4,industrial development of collierville,14,774048.02,661885.66,70132723,2840375.28,-1404441.60,9.26
5,industrial development of germantown,4,237740.75,96081.97,15357250,621968.63,-288145.91,2.15
6,industrial development of millington,1,10280.12,79383.24,7840320,317532.96,-227869.60,0.58
7,health education board memphis,73,930606.59,0.00,128138985,5189628.89,-4259022.30,6.00
8,health education board shelby,8,133533.27,0.00,27687400,1121339.70,-987806.43,0.86
9,miscellaneous contracts,10,35460.00,0.00,9033520,365857.56,-330397.56,0.23


In [117]:
# Check the data types of each variable
df.dtypes

board                                       object
number_of_contracts                          int64
amount_billed_pilot_inl                     object
amount_billed_debt_service                  object
assessment                                  object
tax_based_on_assessment_at_2021_tax_rate    object
difference                                  object
percentage_total_pilot_billing              object
dtype: object

In [118]:
# Fix the data type of each variable
names_to_types = {"board": str, 
                  "number_of_contracts": int, 
                  "amount_billed_pilot_inl": float, 
                  "amount_billed_debt_service": float, 
                  "assessment": float,
                  "tax_based_on_assessment_at_2021_tax_rate": float, 
                  "difference": float,
                  "percentage_total_pilot_billing": float,
                  }
df = cast_data_types(df, names_to_types)
df.dtypes

board                                        object
number_of_contracts                           int64
amount_billed_pilot_inl                     float64
amount_billed_debt_service                  float64
assessment                                  float64
tax_based_on_assessment_at_2021_tax_rate    float64
difference                                  float64
percentage_total_pilot_billing              float64
dtype: object

In [119]:
# Create a unique index column based on the values in the identifying column
# Note: Ids are based on a deterministic hash, which means they are 
# reproducible even if the function is run by a different user at a different time,
# as long as the string in the identifying column is the same. 

identifying_columns = ["board"]
df = set_unique_index(df=df, columns_to_hash = identifying_columns, index_length_limit=12)
df

,board,number_of_contracts,amount_billed_pilot_inl,amount_billed_debt_service,assessment,tax_based_on_assessment_at_2021_tax_rate,difference,percentage_total_pilot_billing
id,,,,,,,,
5b1f4c0c65a3,downtown memphis commission,123,1981904.47,1226006.18,349460540.0,14153151.87,-10945241.22,20.69
d5fe3a85ec4e,edge shelby,243,2856659.78,6149156.86,673220031.0,27265411.26,-18259594.62,58.08
abb34861ca91,industrial development of arlington,15,52116.90,114566.56,11977950.0,485106.98,-318423.52,1.07
aada4183f85c,industrial development of bartlett,11,15257.82,146630.61,14962783.0,605992.71,-444104.28,1.04
91e543f3085f,industrial development of collierville,14,774048.02,661885.66,70132723.0,2840375.28,-1404441.60,9.26
3f1cf0abc31c,industrial development of germantown,4,237740.75,96081.97,15357250.0,621968.63,-288145.91,2.15
757a7152194c,industrial development of millington,1,10280.12,79383.24,7840320.0,317532.96,-227869.60,0.58
3bcfcbd3a12d,health education board memphis,73,930606.59,0.00,128138985.0,5189628.89,-4259022.30,6.00
64c9e33b120e,health education board shelby,8,133533.27,0.00,27687400.0,1121339.70,-987806.43,0.86


In [123]:
# Save the now-clean csv and convert it to utf-8 format
outfile_path = "../data/processed/s1_2019.csv"
df.to_csv(outfile_path)
convert_utf8(original_file_path=outfile_path, new_file_path=outfile_path)